<a href="https://colab.research.google.com/github/WideSu/Python-for-DS/blob/main/Compare_HyperParam_Tuning_Methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TO-DO
- [x] Test the average time usage and RMSE for each epoch using scikit-learn random search
- [ ] Test TPE hyper param tuning for HyperOpt, Ray, Optuna
- [ ] Plot the RMSE through timeline
- [ ] Use the different sampler in Optuna: Random,TPE,CMA-ES,NSGA-II

The outcome:
- A chart consisting the average RMSE and excuation time for all hyper parameter tunning methods

|HPO Package                                  |Avg RMSE                        |Avg Time Ellipsed                                            |
|---------------------------------------------|--------------------------------|-------------------------------------------------------------|
|Scikit-learn                                 |                                |                                                             |
|HyperOpt                                     |                                |                                                             |
|Ray                                          |                                |                                                             |
|Optuna                                       |                                |                                                             |

- One time series plot

<img src="https://user-images.githubusercontent.com/44923423/171923215-292e776a-79aa-4a08-8e81-a2ef627bd42a.png" data-canonical-src="https://user-images.githubusercontent.com/44923423/171923215-292e776a-79aa-4a08-8e81-a2ef627bd42a.png" width="500" height="300" />


|Library|Pros|Cons|Scenario|
|-|-|-|-|
|Scikit-learn|Flexible and basic|Only 2 basic methods (grid/random), New methods are not stable|Tradictional tuning|
|HyperOpt|High-speed and flexible,New search method: TPE/ATPE| Out-of-date interface |Time-limited|
|Ray|Systematic and well wrapped|Too customized and not flexible,Time-cost on initialization|Fast development and deployment with various tuning methods|
|Optuna|Well-performed and light;Include all popular and stable tuning methods |Not well wrapped for all methods|Accurate, flexible required|


In [ ]:
# @title Mont on Google Drive
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/HPO/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/HPO


In [ ]:
# @title Install and import packages
! pip install dateutil
! pip install lightgbm
! pip install optuna
import pandas as pd
import dateutil
import datetime
import optuna
from tqdm import tqdm, trange
from lightgbm import LGBMRegressor
import sklearn
import math
import time

In [ ]:
# @title Read-in data and check data type and volume
df = pd.read_csv('./exp_data.csv')
df.info()

In [ ]:
# @title Change into datatime type
df[["date"]] = df[["date"]].apply(pd.to_datetime)

In [ ]:
library_evaluation_df = {
    'Library' : [],
    'Train Start Date': [],
    'Train End Date': [],
    'Test Start Date': [],
    'Test End Date': [],
    'Smallest RMSE': [],
    'Time Ellipsed': []
}

In [ ]:
# @title optuna hyper param tuning
# Configuration 
train_timespan_months = 180
whole_period_months = 60
test_timespan_months = 1
first_end_time = datetime.datetime(2015, 12, 1)
feat_cols = ['absacc', 'acc', 'age', 'agr', 'baspread','bm', 'bm_ia',
             'cash', 'cashdebt', 'cashpr', 'cfp', 'cfp_ia', 'chatoia', 'chcsho', 'chempia', 'chinv', 'chmom',
             'chpmia', 'chtx', 'cinvest', 'convind', 'currat', 'depr', 'divi', 'divo', 'dolvol', 'dy', 
             'egr', 'ep', 'gma', 'grcapx', 'grltnoa', 'herf', 'hire', 'ill', 'indmom', 'invest', 'lev', 'lgr',
             'maxret', 'mom12m', 'mom1m', 'mom36m', 'mom6m', 'ms', 'mve_ia', 'mvel1', 'nincr', 'operprof',
             'orgcap', 'pchcapx_ia', 'pchcurrat', 'pchdepr', 'pchgm_pchsale', 'pchquick', 'pchsale_pchinvt',
             'pchsale_pchrect', 'pchsale_pchxsga', 'pchsaleinv', 'pctacc', 'ps', 'quick', 'rd', 'rd_mve',
             'rd_sale', 'realestate', 'retvol', 'roaq', 'roavol', 'roeq', 'roic', 'rsup', 'salecash', 'pricedelay',
             'saleinv', 'salerec', 'secured', 'securedind', 'sgr', 'sin', 'sp', 'std_dolvol', 'std_turn',
             'stdacc', 'stdcf', 'tang', 'tb', 'turn', 'zerotrade','aeavol','ear','beta','betasq','idiovol']
y_col = 'ret'
train_end_date = first_end_time
time_usage = []
score_list = []
timeline = []

# Evaluation details for each train and test timespan
evaluate_detail_df = {
    'Train Start Date': [],
    'Train End Date': [],
    'Test Start Date': [],
    'Test End Date': [],
    'Smallest RMSE': [],
    'Time Ellipsed': []
}
predict_times = 60
for period_time in trange(predict_times):
    train_start_date = train_end_date - dateutil.relativedelta.relativedelta(months=train_timespan_months)
    test_end_date = train_end_date + dateutil.relativedelta.relativedelta(months=test_timespan_months)
    print(train_start_date, train_end_date, test_end_date)
    train_data = df.query(f'"{train_start_date}" < date <= "{train_end_date}"')
    test_data = df.query(f'"{train_end_date}" < date <= "{test_end_date}"')
    X_train = train_data[feat_cols].values
    y_train = train_data[y_col].values
    X_test = test_data[feat_cols].values
    y_test = test_data[y_col].values.ravel()
    X_test = test_data[feat_cols].values
    y_test = test_data[y_col].values.ravel()
    study = optuna.create_study(sampler=optuna.samplers.TPESampler())  # Create a new study.
    def objective(trial):
        param = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),   
        'num_leaves': trial.suggest_int('num_leaves', 10, 512),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 80),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.0, 1.0), # subsample
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),  # eta
        'lambda_l1': trial.suggest_float('lambda_l1', 0.01, 1),  # reg_alpha
        'lambda_l2': trial.suggest_float('lambda_l2', 0.01, 1), # reg_lambda
        }
        model = LGBMRegressor(seed=42, **param)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        mse = sklearn.metrics.mean_squared_error(y_test, y_pred)
        rmse = math.sqrt(mse)
        return rmse  # An objective value linked with the Trial object.
    ts = time.time()
    study.optimize(objective, n_trials=1)  # Invoke optimization of the objective function.
    te = time.time()
    exc_time = te-ts
    evaluate_detail_df['Smallest RMSE'].append(study.best_value)
    evaluate_detail_df['Time Ellipsed'].append(exc_time)
    evaluate_detail_df['Train Start Date'].append(train_start_date)
    evaluate_detail_df['Train End Date'].append(train_end_date)
    evaluate_detail_df['Test Start Date'].append(train_end_date+dateutil.relativedelta.relativedelta(months=1))
    evaluate_detail_df['Test End Date'].append(test_end_date)
    train_end_date += dateutil.relativedelta.relativedelta(months=1)
evaluate_detail_df = pd.DataFrame(evaluate_detail_df)

  0%|          | 0/60 [00:00<?, ?it/s][I 2022-06-03 17:32:45,287] A new study created in memory with name: no-name-0fdd27fe-29f4-4e18-bd90-e385c9590cf7


2000-12-01 00:00:00 2015-12-01 00:00:00 2016-01-01 00:00:00


[I 2022-06-03 17:33:44,046] Trial 0 finished with value: 0.12014407101854203 and parameters: {'n_estimators': 268, 'num_leaves': 372, 'min_data_in_leaf': 26, 'bagging_fraction': 0.974294625529873, 'learning_rate': 0.051771955019260726, 'lambda_l1': 0.3197602119136842, 'lambda_l2': 0.3139139070581193}. Best is trial 0 with value: 0.12014407101854203.
  2%|▏         | 1/60 [00:58<57:54, 58.90s/it][I 2022-06-03 17:33:44,103] A new study created in memory with name: no-name-65861a4c-4122-488c-a4f0-e8217ac3b9a7


2001-01-01 00:00:00 2016-01-01 00:00:00 2016-02-01 00:00:00


[I 2022-06-03 17:35:16,232] Trial 0 finished with value: 0.08019383980045353 and parameters: {'n_estimators': 401, 'num_leaves': 444, 'min_data_in_leaf': 20, 'bagging_fraction': 0.2717637088424665, 'learning_rate': 0.05733894722006798, 'lambda_l1': 0.5653191869972338, 'lambda_l2': 0.8047878837165326}. Best is trial 0 with value: 0.08019383980045353.
  3%|▎         | 2/60 [02:31<1:15:51, 78.48s/it][I 2022-06-03 17:35:16,289] A new study created in memory with name: no-name-c30b1107-40ae-4b49-9a27-f0e0e08cf214


...

[I 2022-06-03 18:13:12,005] Trial 0 finished with value: 0.08473459445233832 and parameters: {'n_estimators': 367, 'num_leaves': 490, 'min_data_in_leaf': 33, 'bagging_fraction': 0.47112341979694927, 'learning_rate': 0.01893882822917211, 'lambda_l1': 0.3548998999783392, 'lambda_l2': 0.244676063140908}. Best is trial 0 with value: 0.08473459445233832.
 97%|█████████▋| 58/60 [40:26<02:03, 61.72s/it][I 2022-06-03 18:13:12,062] A new study created in memory with name: no-name-812a578e-fffb-4955-8756-eb91fd07fd61


2005-10-01 00:00:00 2020-10-01 00:00:00 2020-11-01 00:00:00


[I 2022-06-03 18:13:33,090] Trial 0 finished with value: 0.16823991270667032 and parameters: {'n_estimators': 239, 'num_leaves': 123, 'min_data_in_leaf': 43, 'bagging_fraction': 0.3856901931840869, 'learning_rate': 0.06451953053786437, 'lambda_l1': 0.4671681551781985, 'lambda_l2': 0.7455652397075785}. Best is trial 0 with value: 0.16823991270667032.
 98%|█████████▊| 59/60 [40:47<00:49, 49.53s/it][I 2022-06-03 18:13:33,146] A new study created in memory with name: no-name-a2a59e1f-a306-485e-89e0-2525bf43215b


2005-11-01 00:00:00 2020-11-01 00:00:00 2020-12-01 00:00:00


[I 2022-06-03 18:14:24,296] Trial 0 finished with value: 0.06104910464372973 and parameters: {'n_estimators': 326, 'num_leaves': 237, 'min_data_in_leaf': 47, 'bagging_fraction': 0.7353679216692118, 'learning_rate': 0.01659753603452322, 'lambda_l1': 0.17769560328863834, 'lambda_l2': 0.04840970419390444}. Best is trial 0 with value: 0.06104910464372973.
100%|██████████| 60/60 [41:39<00:00, 41.65s/it]


In [ ]:
evaluate_detail_df

,Train Start Date,Train End Date,Test Start Date,Test End Date,Smallest RMSE,Time Ellipsed
0,2000-12-01,2015-12-01,2016-01-01,2016-01-01,0.120144,58.760128
1,2001-01-01,2016-01-01,2016-02-01,2016-02-01,0.080194,92.129778
2,2001-02-01,2016-02-01,2016-03-01,2016-03-01,0.103784,71.986675
3,2001-03-01,2016-03-01,2016-04-01,2016-04-01,0.074915,27.287619
4,2001-04-01,2016-04-01,2016-05-01,2016-05-01,0.058224,17.130490
5,2001-05-01,2016-05-01,2016-06-01,2016-06-01,0.067073,10.279614
6,2001-06-01,2016-06-01,2016-07-01,2016-07-01,0.068623,66.076798
7,2001-07-01,2016-07-01,2016-08-01,2016-08-01,0.066195,18.987839
8,2001-08-01,2016-08-01,2016-09-01,2016-09-01,0.053899,13.782289
9,2001-09-01,2016-09-01,2016-10-01,2016-10-01,0.077368,8.687830


In [ ]:
library_evaluation_df['Library'].extend(['Optuna' for _ in range(len(evaluate_detail_df))])
library_evaluation_df['Train Start Date'].extend(evaluate_detail_df['Train Start Date'])
library_evaluation_df['Train End Date'].extend(evaluate_detail_df['Train End Date'])
library_evaluation_df['Test Start Date'].extend(evaluate_detail_df['Test Start Date'])
library_evaluation_df['Test End Date'].extend(evaluate_detail_df['Test End Date'])
library_evaluation_df['Smallest RMSE'].extend(evaluate_detail_df['Smallest RMSE'])
library_evaluation_df['Time Ellipsed'].extend(evaluate_detail_df['Time Ellipsed'])

In [ ]:
pd.DataFrame(library_evaluation_df)

,Library,Train Start Date,Train End Date,Test Start Date,Test End Date,Smallest RMSE,Time Ellipsed
0,Optuna,2000-12-01,2015-12-01,2016-01-01,2016-01-01,0.120144,58.760128
1,Optuna,2001-01-01,2016-01-01,2016-02-01,2016-02-01,0.080194,92.129778
2,Optuna,2001-02-01,2016-02-01,2016-03-01,2016-03-01,0.103784,71.986675
3,Optuna,2001-03-01,2016-03-01,2016-04-01,2016-04-01,0.074915,27.287619
4,Optuna,2001-04-01,2016-04-01,2016-05-01,2016-05-01,0.058224,17.130490
5,Optuna,2001-05-01,2016-05-01,2016-06-01,2016-06-01,0.067073,10.279614
6,Optuna,2001-06-01,2016-06-01,2016-07-01,2016-07-01,0.068623,66.076798
7,Optuna,2001-07-01,2016-07-01,2016-08-01,2016-08-01,0.066195,18.987839
8,Optuna,2001-08-01,2016-08-01,2016-09-01,2016-09-01,0.053899,13.782289
9,Optuna,2001-09-01,2016-09-01,2016-10-01,2016-10-01,0.077368,8.687830


# Scikit-learn

Refered to [Scikit-learn RandomnizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)

In [ ]:
# @title Scikit-learn
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
# @title optuna hyper param tuning
# Configuration 
train_timespan_months = 180
whole_period_months = 60
test_timespan_months = 1
first_end_time = datetime.datetime(2015, 12, 1)
feat_cols = ['absacc', 'acc', 'age', 'agr', 'baspread','bm', 'bm_ia',
             'cash', 'cashdebt', 'cashpr', 'cfp', 'cfp_ia', 'chatoia', 'chcsho', 'chempia', 'chinv', 'chmom',
             'chpmia', 'chtx', 'cinvest', 'convind', 'currat', 'depr', 'divi', 'divo', 'dolvol', 'dy', 
             'egr', 'ep', 'gma', 'grcapx', 'grltnoa', 'herf', 'hire', 'ill', 'indmom', 'invest', 'lev', 'lgr',
             'maxret', 'mom12m', 'mom1m', 'mom36m', 'mom6m', 'ms', 'mve_ia', 'mvel1', 'nincr', 'operprof',
             'orgcap', 'pchcapx_ia', 'pchcurrat', 'pchdepr', 'pchgm_pchsale', 'pchquick', 'pchsale_pchinvt',
             'pchsale_pchrect', 'pchsale_pchxsga', 'pchsaleinv', 'pctacc', 'ps', 'quick', 'rd', 'rd_mve',
             'rd_sale', 'realestate', 'retvol', 'roaq', 'roavol', 'roeq', 'roic', 'rsup', 'salecash', 'pricedelay',
             'saleinv', 'salerec', 'secured', 'securedind', 'sgr', 'sin', 'sp', 'std_dolvol', 'std_turn',
             'stdacc', 'stdcf', 'tang', 'tb', 'turn', 'zerotrade','aeavol','ear','beta','betasq','idiovol']
y_col = 'ret'
train_end_date = first_end_time
time_usage = []
score_list = []
timeline = []

# Evaluation details for each train and test timespan
evaluate_detail_df = {
    'Train Start Date': [],
    'Train End Date': [],
    'Test Start Date': [],
    'Test End Date': [],
    'Smallest RMSE': [],
    'Time Ellipsed': []
}
predict_times = 60
def rmse(reg, X, y):
      y_pred = reg.predict(X)
      mse = sklearn.metrics.mean_squared_error(y, y_pred)      
      return math.sqrt(mse)
for period_time in trange(predict_times):
    train_start_date = train_end_date - dateutil.relativedelta.relativedelta(months=train_timespan_months)
    test_end_date = train_end_date + dateutil.relativedelta.relativedelta(months=test_timespan_months)
    print(train_start_date, train_end_date, test_end_date)
    train_data = df.query(f'"{train_start_date}" < date <= "{train_end_date}"')
    test_data = df.query(f'"{train_end_date}" < date <= "{test_end_date}"')
    X_train = train_data[feat_cols].values
    y_train = train_data[y_col].values
    X_test = test_data[feat_cols].values
    y_test = test_data[y_col].values.ravel()
    X_test = test_data[feat_cols].values
    y_test = test_data[y_col].values.ravel()
    model = LGBMRegressor(seed=42)
    param_distribution = dict(
        n_estimators = uniform(loc=50, scale=500),   
        num_leaves = uniform(loc=10, scale=512),
        min_data_in_leaf = uniform( loc=10, scale=80),
        bagging_fraction= uniform( loc=0, scale=0.1), # subsample
        learning_rate= uniform( loc=0.01, scale=0.1),  # eta
        lambda_l1= uniform( loc=0.01, scale=1),  # reg_alpha
        lambda_l2= uniform( loc=0.01, scale=1), # reg_lambda
    )
    search_cv = RandomizedSearchCV(model, 
                                   param_distribution,
                                   scoring=rmse,
                                   random_state=0)
    ts = time.time()
    search_cv.fit(X_train, y_train)
    te = time.time()
    exc_time = te-ts
    evaluate_detail_df['Smallest RMSE'].append(search_cv.best_score_)
    evaluate_detail_df['Time Ellipsed'].append(exc_time)
    evaluate_detail_df['Train Start Date'].append(train_start_date)
    evaluate_detail_df['Train End Date'].append(train_end_date)
    evaluate_detail_df['Test Start Date'].append(train_end_date+dateutil.relativedelta.relativedelta(months=1))
    evaluate_detail_df['Test End Date'].append(test_end_date)
    train_end_date += dateutil.relativedelta.relativedelta(months=1)
evaluate_detail_df = pd.DataFrame(evaluate_detail_df)

  0%|          | 0/60 [00:00<?, ?it/s]

2000-12-01 00:00:00 2015-12-01 00:00:00 2016-01-01 00:00:00


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
50 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py", line 197, in train
    booster = Booster(params=params, train_set=train_set)
  File "/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py", line 1552, in __init__
    train_set.construct().handle,
  File "/usr/local/lib/python3.7/dist-packages/lightgbm/basic.py", line 1001, in construct
    categorical_feature=self.categorical_feature, params=self.params)
  File "/usr/local/lib/python3.7/dist-packages/lightgbm

NotFittedError: ignored